In [3]:
!pip install catboost lightgbm

In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [7]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [8]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [9]:
CatBoost = CatBoostClassifier(cat_features = columns)

In [10]:
target = df["target"]

In [11]:
df_new = df_new.fillna(0)

In [12]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [13]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [14]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [15]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,1334.0,1440.0,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,1706.0,2016.0,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,1323.0,1507.0,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,2039.0,2238.0,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,1628.0,1744.0,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [16]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.264529
0:	learn: 0.6242233	total: 2.88s	remaining: 47m 55s
1:	learn: 0.5886849	total: 5.17s	remaining: 43m
2:	learn: 0.5616931	total: 6.92s	remaining: 38m 19s
3:	learn: 0.5505202	total: 8.9s	remaining: 36m 57s
4:	learn: 0.5442888	total: 11s	remaining: 36m 25s
5:	learn: 0.5354140	total: 12.3s	remaining: 33m 57s
6:	learn: 0.5316674	total: 14.2s	remaining: 33m 31s
7:	learn: 0.5278158	total: 16.3s	remaining: 33m 41s
8:	learn: 0.5263124	total: 18.2s	remaining: 33m 18s
9:	learn: 0.5224016	total: 20.2s	remaining: 33m 16s
10:	learn: 0.5212857	total: 22.4s	remaining: 33m 31s
11:	learn: 0.5194600	total: 24.3s	remaining: 33m 22s
12:	learn: 0.5179490	total: 27s	remaining: 34m 11s
13:	learn: 0.5147156	total: 29.1s	remaining: 34m 10s
14:	learn: 0.5134716	total: 31.3s	remaining: 34m 16s
15:	learn: 0.5115268	total: 33.3s	remaining: 34m 9s
16:	learn: 0.5092358	total: 34.3s	remaining: 33m 6s
17:	learn: 0.5076746	total: 36.5s	remaining: 33m 10s
18:	learn: 0.5066913	total: 38.5s	rem

153:	learn: 0.4621132	total: 5m 31s	remaining: 30m 20s
154:	learn: 0.4620066	total: 5m 33s	remaining: 30m 19s
155:	learn: 0.4614717	total: 5m 36s	remaining: 30m 18s
156:	learn: 0.4613867	total: 5m 38s	remaining: 30m 15s
157:	learn: 0.4613250	total: 5m 40s	remaining: 30m 12s
158:	learn: 0.4604318	total: 5m 42s	remaining: 30m 9s
159:	learn: 0.4603662	total: 5m 44s	remaining: 30m 6s
160:	learn: 0.4603073	total: 5m 46s	remaining: 30m 3s
161:	learn: 0.4602457	total: 5m 47s	remaining: 29m 59s
162:	learn: 0.4601748	total: 5m 50s	remaining: 29m 57s
163:	learn: 0.4599541	total: 5m 52s	remaining: 29m 56s
164:	learn: 0.4594271	total: 5m 54s	remaining: 29m 54s
165:	learn: 0.4593757	total: 5m 56s	remaining: 29m 52s
166:	learn: 0.4593250	total: 5m 58s	remaining: 29m 49s
167:	learn: 0.4592353	total: 6m 1s	remaining: 29m 47s
168:	learn: 0.4592030	total: 6m 3s	remaining: 29m 45s
169:	learn: 0.4588840	total: 6m 5s	remaining: 29m 42s
170:	learn: 0.4588385	total: 6m 7s	remaining: 29m 41s
171:	learn: 0.458

303:	learn: 0.4334722	total: 10m 29s	remaining: 24m 1s
304:	learn: 0.4328423	total: 10m 31s	remaining: 23m 59s
305:	learn: 0.4328190	total: 10m 33s	remaining: 23m 57s
306:	learn: 0.4324865	total: 10m 35s	remaining: 23m 54s
307:	learn: 0.4324680	total: 10m 37s	remaining: 23m 52s
308:	learn: 0.4324460	total: 10m 38s	remaining: 23m 48s
309:	learn: 0.4321716	total: 10m 41s	remaining: 23m 46s
310:	learn: 0.4321034	total: 10m 43s	remaining: 23m 44s
311:	learn: 0.4317874	total: 10m 45s	remaining: 23m 43s
312:	learn: 0.4315148	total: 10m 47s	remaining: 23m 40s
313:	learn: 0.4314899	total: 10m 49s	remaining: 23m 38s
314:	learn: 0.4314622	total: 10m 51s	remaining: 23m 36s
315:	learn: 0.4309074	total: 10m 52s	remaining: 23m 33s
316:	learn: 0.4308370	total: 10m 54s	remaining: 23m 30s
317:	learn: 0.4306859	total: 10m 56s	remaining: 23m 28s
318:	learn: 0.4306713	total: 10m 58s	remaining: 23m 25s
319:	learn: 0.4306405	total: 11m	remaining: 23m 23s
320:	learn: 0.4305709	total: 11m 2s	remaining: 23m 21

451:	learn: 0.4155362	total: 15m 20s	remaining: 18m 35s
452:	learn: 0.4155266	total: 15m 22s	remaining: 18m 33s
453:	learn: 0.4152723	total: 15m 24s	remaining: 18m 31s
454:	learn: 0.4152604	total: 15m 26s	remaining: 18m 29s
455:	learn: 0.4152457	total: 15m 28s	remaining: 18m 27s
456:	learn: 0.4152282	total: 15m 30s	remaining: 18m 25s
457:	learn: 0.4151945	total: 15m 31s	remaining: 18m 22s
458:	learn: 0.4151494	total: 15m 34s	remaining: 18m 21s
459:	learn: 0.4147301	total: 15m 35s	remaining: 18m 18s
460:	learn: 0.4147248	total: 15m 37s	remaining: 18m 16s
461:	learn: 0.4143444	total: 15m 40s	remaining: 18m 14s
462:	learn: 0.4143357	total: 15m 42s	remaining: 18m 12s
463:	learn: 0.4143270	total: 15m 44s	remaining: 18m 10s
464:	learn: 0.4141433	total: 15m 46s	remaining: 18m 8s
465:	learn: 0.4141349	total: 15m 48s	remaining: 18m 6s
466:	learn: 0.4140989	total: 15m 49s	remaining: 18m 4s
467:	learn: 0.4140834	total: 15m 51s	remaining: 18m 1s
468:	learn: 0.4140626	total: 15m 53s	remaining: 17m 

599:	learn: 0.4002370	total: 20m 10s	remaining: 13m 27s
600:	learn: 0.4002236	total: 20m 12s	remaining: 13m 24s
601:	learn: 0.4001985	total: 20m 14s	remaining: 13m 22s
602:	learn: 0.3998221	total: 20m 16s	remaining: 13m 20s
603:	learn: 0.3998124	total: 20m 18s	remaining: 13m 18s
604:	learn: 0.3997717	total: 20m 20s	remaining: 13m 16s
605:	learn: 0.3990400	total: 20m 22s	remaining: 13m 14s
606:	learn: 0.3990267	total: 20m 24s	remaining: 13m 13s
607:	learn: 0.3989740	total: 20m 26s	remaining: 13m 11s
608:	learn: 0.3989624	total: 20m 28s	remaining: 13m 8s
609:	learn: 0.3989517	total: 20m 30s	remaining: 13m 6s
610:	learn: 0.3989291	total: 20m 32s	remaining: 13m 4s
611:	learn: 0.3989109	total: 20m 34s	remaining: 13m 2s
612:	learn: 0.3988772	total: 20m 35s	remaining: 13m
613:	learn: 0.3988644	total: 20m 37s	remaining: 12m 58s
614:	learn: 0.3988481	total: 20m 39s	remaining: 12m 55s
615:	learn: 0.3988413	total: 20m 41s	remaining: 12m 53s
616:	learn: 0.3988208	total: 20m 43s	remaining: 12m 52s


748:	learn: 0.3916524	total: 25m 2s	remaining: 8m 23s
749:	learn: 0.3916423	total: 25m 4s	remaining: 8m 21s
750:	learn: 0.3913693	total: 25m 6s	remaining: 8m 19s
751:	learn: 0.3913341	total: 25m 8s	remaining: 8m 17s
752:	learn: 0.3913166	total: 25m 10s	remaining: 8m 15s
753:	learn: 0.3913020	total: 25m 12s	remaining: 8m 13s
754:	learn: 0.3912513	total: 25m 14s	remaining: 8m 11s
755:	learn: 0.3912416	total: 25m 17s	remaining: 8m 9s
756:	learn: 0.3912278	total: 25m 19s	remaining: 8m 7s
757:	learn: 0.3912237	total: 25m 22s	remaining: 8m 5s
758:	learn: 0.3912149	total: 25m 24s	remaining: 8m 4s
759:	learn: 0.3912041	total: 25m 26s	remaining: 8m 2s
760:	learn: 0.3911852	total: 25m 28s	remaining: 8m
761:	learn: 0.3911705	total: 25m 30s	remaining: 7m 58s
762:	learn: 0.3911651	total: 25m 32s	remaining: 7m 56s
763:	learn: 0.3910732	total: 25m 34s	remaining: 7m 53s
764:	learn: 0.3910366	total: 25m 36s	remaining: 7m 51s
765:	learn: 0.3910229	total: 25m 38s	remaining: 7m 49s
766:	learn: 0.3910162	t

899:	learn: 0.3832732	total: 30m 12s	remaining: 3m 21s
900:	learn: 0.3832228	total: 30m 14s	remaining: 3m 19s
901:	learn: 0.3832158	total: 30m 17s	remaining: 3m 17s
902:	learn: 0.3832108	total: 30m 19s	remaining: 3m 15s
903:	learn: 0.3829167	total: 30m 21s	remaining: 3m 13s
904:	learn: 0.3829083	total: 30m 23s	remaining: 3m 11s
905:	learn: 0.3825235	total: 30m 25s	remaining: 3m 9s
906:	learn: 0.3825122	total: 30m 27s	remaining: 3m 7s
907:	learn: 0.3825037	total: 30m 29s	remaining: 3m 5s
908:	learn: 0.3824975	total: 30m 31s	remaining: 3m 3s
909:	learn: 0.3823482	total: 30m 32s	remaining: 3m 1s
910:	learn: 0.3823360	total: 30m 34s	remaining: 2m 59s
911:	learn: 0.3823189	total: 30m 36s	remaining: 2m 57s
912:	learn: 0.3821925	total: 30m 38s	remaining: 2m 55s
913:	learn: 0.3821855	total: 30m 40s	remaining: 2m 53s
914:	learn: 0.3821750	total: 30m 42s	remaining: 2m 51s
915:	learn: 0.3821619	total: 30m 44s	remaining: 2m 49s
916:	learn: 0.3821430	total: 30m 46s	remaining: 2m 47s
917:	learn: 0.3

In [17]:
test = test.fillna(0)

In [18]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [19]:
test = test.drop("TailNum", axis = 1)

In [20]:
test = test.drop("Id", axis = 1)

In [21]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [22]:
proba = CatBoost.predict_proba(test)[:, 1]

In [23]:
submission = pd.read_csv("sample_submission.csv")

In [24]:
submission['target'] = proba

In [25]:
submission.to_csv("submission", index = False)